In [2]:
import pandas as pd
hosp = r"C:\Users\AISWARYA\Desktop\display pics\mimic\mimic data\mimic-iv-3.1\hosp"
icu  = r"C:\Users\AISWARYA\Desktop\display pics\mimic\mimic data\mimic-iv-3.1\icu"
chunks = pd.read_csv(f"{hosp}\\labevents.csv",
                     usecols=['subject_id','hadm_id','itemid','charttime','valuenum'],
                     chunksize=500000)
creatinine_list = [c[c['itemid']==50912] for c in chunks]
creatinine = pd.concat(creatinine_list)
creatinine['charttime'] = pd.to_datetime(creatinine['charttime'])
creatinine.sort_values(['subject_id','charttime'], inplace=True)
def aki_flag(g):
    g = g.sort_values('charttime')
    base = g['valuenum'].min()
    g['AKI_flag'] = ((g['valuenum'].diff() >= 0.3) |
                     (g['valuenum'] >= 1.5 * base)).astype(int)
    return g
creat_aki = creatinine.groupby('subject_id', group_keys=False).apply(aki_flag)
creat_aki = creat_aki[creat_aki['AKI_flag']==1]
creat_aki.head()

C:\Users\AISWARYA\AppData\Local\Temp\ipykernel_16756\3390544405.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  creat_aki = creatinine.groupby('subject_id', group_keys=False).apply(aki_flag)


,subject_id,hadm_id,itemid,charttime,valuenum,AKI_flag
242,10000032,NaN,50912,2180-06-26 16:10:00,0.6,1
352,10000032,NaN,50912,2180-07-23 06:39:00,0.7,1
397,10000032,29079034.0,50912,2180-07-23 21:45:00,0.5,1
466,10000032,NaN,50912,2180-07-27 08:30:00,0.6,1
487,10000032,NaN,50912,2180-08-05 21:20:00,0.6,1


In [3]:
icustays = pd.read_csv(f"{icu}\\icustays.csv",
                       usecols=['subject_id','hadm_id','stay_id','intime','outtime'])
icustays[['intime','outtime']] = icustays[['intime','outtime']].apply(pd.to_datetime)
creat_aki = creat_aki.merge(icustays, on=['subject_id','hadm_id'], how='inner')
creat_aki = creat_aki[
    (creat_aki['charttime'] >= creat_aki['intime']) &
    (creat_aki['charttime'] <= creat_aki['outtime'])
]


In [4]:
urine = pd.read_csv(f"{icu}\\outputevents.csv",
                    usecols=['subject_id','hadm_id','stay_id','charttime','itemid','value','valueuom'])
urine['charttime'] = pd.to_datetime(urine['charttime'])
urine.rename(columns={'value':'urine_ml'}, inplace=True)
urine['valueuom'] = urine['valueuom'].str.lower()
urine.loc[urine['valueuom']=='l', 'urine_ml'] *= 1000
urine_ids = [226559,227488,40055,40056,40057,40058,40061,40062,40063,40065,40069,40085,40094]
urine = urine[urine['itemid'].isin(urine_ids)]
urine['date'] = urine['charttime'].dt.date
urine_daily = (
    urine.groupby(['subject_id','hadm_id','stay_id','date'])['urine_ml']
    .sum().reset_index()
    .rename(columns={'urine_ml':'urine_total_ml'})
)


In [5]:
vaso = pd.read_csv(f"{icu}\\inputevents.csv",
                   usecols=['subject_id','hadm_id','stay_id','starttime','endtime','itemid','amount','amountuom'],
                   low_memory=False)
vaso = vaso[vaso['itemid']==222315]   # Vasopressin
vaso[['starttime','endtime']] = vaso[['starttime','endtime']].apply(pd.to_datetime)
vaso['date'] = vaso['starttime'].dt.date
vaso_daily = (
    vaso.groupby(['subject_id','hadm_id','stay_id','date'])['amount']
    .sum().reset_index()
    .rename(columns={'amount':'vasopressin_dose'})
)
vaso_daily['on_vasopressin'] = True


In [6]:
icu_daily = pd.merge(
    urine_daily, vaso_daily,
    on=['subject_id','hadm_id','stay_id','date'],
    how='left'
)
icu_daily['on_vasopressin'] = icu_daily['on_vasopressin'].fillna(False)
final_cohort = pd.merge(
    creat_aki[['subject_id','hadm_id','stay_id','AKI_flag','valuenum','charttime']],
    icu_daily,
    on=['subject_id','hadm_id','stay_id'],
    how='left'
)

final_cohort['chartdate'] = final_cohort['charttime'].dt.date


C:\Users\AISWARYA\AppData\Local\Temp\ipykernel_16756\2097405235.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  icu_daily['on_vasopressin'] = icu_daily['on_vasopressin'].fillna(False)


In [8]:
print("AKI ICU patients:", final_cohort['subject_id'].nunique())
final_cohort.to_csv(r"C:\Users\Aiswarya\Desktop\MIMIC_AKI_ICU_Cohort.csv", index=False)


AKI ICU patients: 33187


In [ ]:
final_cohort.head()

,subject_id,hadm_id,stay_id,AKI_flag,valuenum,charttime,date,urine_total_ml,vasopressin_dose,on_vasopressin,chartdate
0,10000032,29079034.0,39553978,1,0.5,2180-07-23 21:45:00,NaN,NaN,NaN,NaN,2180-07-23
1,10000690,25860671.0,37081114,1,0.9,2150-11-03 02:56:00,2150-11-02,170.0,NaN,False,2150-11-03
2,10000690,25860671.0,37081114,1,0.9,2150-11-03 02:56:00,2150-11-03,635.0,NaN,False,2150-11-03
3,10000690,25860671.0,37081114,1,0.9,2150-11-03 02:56:00,2150-11-04,2515.0,NaN,False,2150-11-03
4,10000690,25860671.0,37081114,1,0.9,2150-11-03 02:56:00,2150-11-05,2685.0,NaN,False,2150-11-03


In [10]:
patient_counts = final_cohort['subject_id'].value_counts()
multi_record_patients = patient_counts[patient_counts >= 2]
print(f" Patients with ≥2 records: {len(multi_record_patients)}")
print(f" Total unique patients in cohort: {final_cohort['subject_id'].nunique()}")
print("\nExample patients with multiple entries:")
print(multi_record_patients.head(10))


👩‍⚕️ Patients with ≥2 records: 30404
📊 Total unique patients in cohort: 33187

Example patients with multiple entries:
subject_id
14411859    15859
11633737    15580
13697731    11473
10699336     9680
15149227     9368
11311104     9089
16175244     8971
17876390     7982
16454913     7694
13224377     7628
Name: count, dtype: int64


In [11]:

patient_counts = final_cohort['subject_id'].value_counts()
multi_record_patients = patient_counts[patient_counts >= 2].index
recovery_cohort = final_cohort[final_cohort['subject_id'].isin(multi_record_patients)]
print(f"Recovery cohort created: {recovery_cohort.shape[0]} rows")
print(f"Unique patients in recovery cohort: {recovery_cohort['subject_id'].nunique()}")
print(f"Removed patients with only 1 reading: {final_cohort['subject_id'].nunique() - recovery_cohort['subject_id'].nunique()}")


✅ Recovery cohort created: 3056786 rows
👩‍⚕️ Unique patients in recovery cohort: 30404
📉 Removed patients with only 1 reading: 2783


In [9]:
print("Computing recovery metrics, please wait...")
final_cohort['subject_id'] = final_cohort['subject_id'].astype('int32')
final_cohort['valuenum'] = pd.to_numeric(final_cohort['valuenum'], errors='coerce')
final_cohort.sort_values(['subject_id', 'charttime'], inplace=True)
first_creat = final_cohort.groupby('subject_id', observed=True)['valuenum'].first()
last_creat  = final_cohort.groupby('subject_id', observed=True)['valuenum'].last()
final_cohort['first_creat'] = final_cohort['subject_id'].map(first_creat)
final_cohort['last_creat']  = final_cohort['subject_id'].map(last_creat)
final_cohort['recovery_ratio'] = final_cohort['first_creat'] / final_cohort['last_creat']
final_cohort['recovery_class'] = pd.cut(
    final_cohort['recovery_ratio'],
    bins=[0, 0.9, 1.2, float('inf')],
    labels=['Worsening', 'Improving', 'Recovered']
)
print("Added recovery_ratio and recovery_class successfully")
print(final_cohort[['subject_id', 'recovery_ratio', 'recovery_class']].head(10))


⏳ Computing recovery metrics, please wait...
✅ Added recovery_ratio and recovery_class successfully!
   subject_id  recovery_ratio recovery_class
0    10000032             1.0      Improving
1    10000690             1.0      Improving
2    10000690             1.0      Improving
3    10000690             1.0      Improving
4    10000690             1.0      Improving
5    10000690             1.0      Improving
6    10000690             1.0      Improving
7    10000690             1.0      Improving
8    10000690             1.0      Improving
9    10000690             1.0      Improving


In [9]:
final_cohort.shape

(3059569, 11)

In [14]:
unique_subjects = final_cohort['subject_id'].nunique()
print("Number of unique AKI ICU patients:", unique_subjects)


Number of unique AKI ICU patients: 33187


In [15]:
final_cohort['date'] = pd.to_datetime(final_cohort['date'])
final_cohort.sort_values(
    by=['subject_id', 'hadm_id', 'stay_id', 'date'],
    ascending=[True, True, True, True],
    inplace=True
)
final_cohort.reset_index(drop=True, inplace=True)
print(" Data sorted successfully")
print(final_cohort[['subject_id','hadm_id','stay_id','date']].head(10))
final_cohort.to_csv(r"C:\Users\Aiswarya\Desktop\MIMIC_AKI_ICU_Cohort_sorted.csv", index=False)
print(" Sorted file saved successfully!")


✅ Data sorted successfully!
   subject_id     hadm_id   stay_id       date
0    10000032  29079034.0  39553978        NaT
1    10000690  25860671.0  37081114 2150-11-02
2    10000690  25860671.0  37081114 2150-11-02
3    10000690  25860671.0  37081114 2150-11-03
4    10000690  25860671.0  37081114 2150-11-03
5    10000690  25860671.0  37081114 2150-11-04
6    10000690  25860671.0  37081114 2150-11-04
7    10000690  25860671.0  37081114 2150-11-05
8    10000690  25860671.0  37081114 2150-11-05
9    10000690  25860671.0  37081114 2150-11-06
💾 Sorted file saved successfully!


In [22]:
final_cohort.head(20)


,subject_id,hadm_id,stay_id,AKI_flag,valuenum,charttime,date,urine_total_ml,vasopressin_dose,on_vasopressin,chartdate,first_creat,last_creat,recovery_ratio,recovery_class
0,10000032,29079034.0,39553978,1,0.5,2180-07-23 21:45:00,NaT,NaN,NaN,NaN,2180-07-23,0.5,0.5,1.000,Improving
1,10000690,25860671.0,37081114,1,0.9,2150-11-03 02:56:00,2150-11-02,170.0,NaN,False,2150-11-03,0.9,0.9,1.000,Improving
2,10000690,25860671.0,37081114,1,0.9,2150-11-04 03:03:00,2150-11-02,170.0,NaN,False,2150-11-04,0.9,0.9,1.000,Improving
3,10000690,25860671.0,37081114,1,0.9,2150-11-03 02:56:00,2150-11-03,635.0,NaN,False,2150-11-03,0.9,0.9,1.000,Improving
4,10000690,25860671.0,37081114,1,0.9,2150-11-04 03:03:00,2150-11-03,635.0,NaN,False,2150-11-04,0.9,0.9,1.000,Improving
5,10000690,25860671.0,37081114,1,0.9,2150-11-03 02:56:00,2150-11-04,2515.0,NaN,False,2150-11-03,0.9,0.9,1.000,Improving
6,10000690,25860671.0,37081114,1,0.9,2150-11-04 03:03:00,2150-11-04,2515.0,NaN,False,2150-11-04,0.9,0.9,1.000,Improving
7,10000690,25860671.0,37081114,1,0.9,2150-11-03 02:56:00,2150-11-05,2685.0,NaN,False,2150-11-03,0.9,0.9,1.000,Improving
8,10000690,25860671.0,37081114,1,0.9,2150-11-04 03:03:00,2150-11-05,2685.0,NaN,False,2150-11-04,0.9,0.9,1.000,Improving
9,10000690,25860671.0,37081114,1,0.9,2150-11-03 02:56:00,2150-11-06,712.0,NaN,False,2150-11-03,0.9,0.9,1.000,Improving


In [23]:
final_cohort.shape

(3059569, 15)

In [24]:
import pandas as pd
df = pd.read_csv(r"C:\Users\AISWARYA\Desktop\final_cohort_cleaned1.csv")
final_summary = (
    df.groupby('subject_id', as_index=False)
      .agg({
          'urine_total_ml': 'mean',         # mean urine output
          'valuenum': 'mean',               # mean creatinine
          'on_vasopressin': 'max',          # if True even once → True
          'vasopressin_dose': 'mean',       # mean dose (adjusted below)
          'first_creat': 'max',             # highest first creatinine
          'last_creat': 'min',              # lowest last creatinine
          'recovery_ratio': 'max',          # best recovery ratio
          'recovery_class': lambda x: x.mode()[0] if not x.mode().empty else None  # most common class
      })
)
final_summary.rename(columns={
    'urine_total_ml': 'urine_output_mean',
    'valuenum': 'creatinine_mean',
    'vasopressin_dose': 'vasopressin_dose_mean'
}, inplace=True)
final_summary.loc[final_summary['on_vasopressin'] == False, 'vasopressin_dose_mean'] = None
final_summary.to_csv(r"C:\Users\AISWARYA\Desktop\final_patient_summary.csv", index=False)

print("Final summarized dataset created and saved as 'final_patient_summary.csv'")
print(final_summary.head())


✅ Final summarized dataset created and saved as 'final_patient_summary.csv'
   subject_id  urine_output_mean  creatinine_mean on_vasopressin  \
0    10000032                NaN         0.500000            NaN   
1    10000690             1343.4         0.900000          False   
2    10001843              107.5         1.700000          False   
3    10001884             1170.8         1.044444          False   
4    10002013             1217.5         1.100000          False   

   vasopressin_dose_mean  first_creat  last_creat  recovery_ratio  \
0                    NaN          0.5         0.5           1.000   
1                    NaN          0.9         0.9           1.000   
2                    NaN          1.7         1.7           1.000   
3                    NaN          1.1         0.8           1.375   
4                    NaN          1.1         1.1           1.000   

  recovery_class  
0      Improving  
1      Improving  
2      Improving  
3      Recovered  
4    

In [25]:
final_summary.shape

(33187, 9)

In [26]:
final_summary['subject_id'].is_unique


True

In [27]:
final_summary = final_summary.dropna(subset=['urine_output_mean'])
print(" Rows after removing empty urine output:", len(final_summary))
print("Remaining unique subjects:", final_summary['subject_id'].nunique())



✅ Rows after removing empty urine output: 27858
Remaining unique subjects: 27858


In [29]:
final_summary.to_csv(r"C:\Users\AISWARYA\Desktop\final_patient_summary_cleaned.csv", index=False)
print(" Cleaned dataset saved successfully at:")
print(r"C:\Users\AISWARYA\Desktop\final_patient_summary_cleaned.csv")



✅ Cleaned dataset saved successfully at:
C:\Users\AISWARYA\Desktop\final_patient_summary_cleaned.csv


In [30]:
final_summary.shape

(27858, 9)

In [20]:

final_cohort.to_csv(r"C:\Users\AISWARYA\Desktop\final_cohort_cleaned1.csv", index=False)

print("File saved successfully at Desktop as 'final_cohort_cleaned.csv'")


✅ File saved successfully at Desktop as 'final_cohort_cleaned.csv'


In [21]:
import pandas as pd
df = pd.read_csv(r"C:\Users\AISWARYA\Desktop\final_cohort_cleaned1.csv")
final_summary = (
    df.groupby('subject_id', as_index=False)
      .agg({
          'urine_total_ml': 'mean',          
          'valuenum': 'mean',                
          'on_vasopressin': 'max',           
          'vasopressin_dose': 'mean'         
      })
)

final_summary = final_summary.rename(columns={
    'urine_total_ml': 'mean_urine_output_ml',
    'valuenum': 'mean_creatinine',
    'vasopressin_dose': 'mean_vasopressin_dose'
})

final_summary.loc[final_summary['on_vasopressin'] == False, 'mean_vasopressin_dose'] = None
final_summary.to_csv(r"C:\Users\AISWARYA\Desktop\final_cohort_patientlevel.csv", index=False)

print("Aggregated file saved successfully at Desktop as 'final_cohort_patientlevel.csv'")
print(final_summary.head())


✅ Aggregated file saved successfully at Desktop as 'final_cohort_patientlevel.csv'
   subject_id  mean_urine_output_ml  mean_creatinine on_vasopressin  \
0    10000032                   NaN         0.500000            NaN   
1    10000690                1343.4         0.900000          False   
2    10001843                 107.5         1.700000          False   
3    10001884                1170.8         1.044444          False   
4    10002013                1217.5         1.100000          False   

   mean_vasopressin_dose  
0                    NaN  
1                    NaN  
2                    NaN  
3                    NaN  
4                    NaN  


In [22]:
summary = final_cohort['on_vasopressin'].value_counts()
print("Vasopressin usage summary:")
print(summary)


📊 Vasopressin usage summary:
on_vasopressin
False    2739649
True      275418
Name: count, dtype: int64


In [23]:
vaso_status_per_patient = (
    final_cohort.groupby('subject_id')['on_vasopressin']
    .apply(lambda x: True if True in x.values
                     else False if all(x.fillna(False) == False)
                     else pd.NA)
    .reset_index()
)
vaso_summary = vaso_status_per_patient['on_vasopressin'].value_counts(dropna=False)
print("📊 Vasopressin usage summary per unique patient:")
print(vaso_summary)


C:\Users\AISWARYA\AppData\Local\Temp\ipykernel_24944\4180034916.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  else False if all(x.fillna(False) == False)


📊 Vasopressin usage summary per unique patient:
on_vasopressin
False    29118
True      4069
Name: count, dtype: int64


In [24]:
unique_patients = final_cohort['subject_id'].nunique()

print(f" Total unique patients in the cohort: {unique_patients}")


🧍‍♀️ Total unique patients in the cohort: 33187


In [27]:
print(final_cohort.columns.tolist())


['subject_id', 'hadm_id', 'stay_id', 'AKI_flag', 'valuenum', 'charttime', 'date', 'urine_total_ml', 'vasopressin_dose', 'on_vasopressin', 'chartdate']


In [28]:
aki_per_patient = final_cohort.groupby('subject_id')['AKI_flag'].max()
print("Unique AKI patients:", aki_per_patient.sum())


Unique AKI patients: 33187
